In [1]:
import time
import os
import requests
import io
import hashlib
from PIL import Image

In [2]:
from selenium import webdriver
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = "./chromedriver"
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

/var/folders/ch/l3hmn88x3sl5tcmrw3l25hrr0000gn/T/ipykernel_13054/1449100853.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=DRIVER_PATH)


In [3]:
wd.get('http://google.com')

search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Thai')

/var/folders/ch/l3hmn88x3sl5tcmrw3l25hrr0000gn/T/ipykernel_13054/308075189.py:3: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  search_box = wd.find_element_by_css_selector('input.gLFyf')


In [5]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [6]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [7]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [10]:
search_term = 'thai art style'

search_and_download(
    search_term=search_term,
    driver_path=DRIVER_PATH
)

/var/folders/ch/l3hmn88x3sl5tcmrw3l25hrr0000gn/T/ipykernel_13054/4098395253.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
/var/folders/ch/l3hmn88x3sl5tcmrw3l25hrr0000gn/T/ipykernel_13054/4128504895.py:19: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


/var/folders/ch/l3hmn88x3sl5tcmrw3l25hrr0000gn/T/ipykernel_13054/4128504895.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 5 image links, done!
SUCCESS - saved https://cdn5.vectorstock.com/i/1000x1000/93/04/thai-art-pattern-old-style-the-wall-temple-vector-1459304.jpg - as ./images/thai_art_style/be3203dab9.jpg
SUCCESS - saved https://thumbs.dreamstime.com/z/thai-gold-painting-art-style-wall-24255018.jpg - as ./images/thai_art_style/d3447247ac.jpg
SUCCESS - saved https://i.pinimg.com/564x/2a/2b/f8/2a2bf817a41c1ac9d66262315eaea2da.jpg - as ./images/thai_art_style/246483e26b.jpg
SUCCESS - saved https://previews.123rf.com/images/swingvoodoo/swingvoodoo1204/swingvoodoo120400335/13118119-thai-art-style-on-wall-take-photo-from-temple-in-ubon-ratchathani-thailand.jpg - as ./images/thai_art_style/51c7be1a63.jpg
SUCCESS - saved https://royalthaiart.com/wp-content/uploads/2020/06/thai-style-painting.jpg - as ./images/thai_art_style/f6abb5f085.jpg


In [25]:
wd.quit()